<a href="https://colab.research.google.com/github/nandodeomkar/BFD-using-ResNet50/blob/main/Full_Fledged_Try_at_BFD_using_ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

In [5]:
import cv2
import os

# Set the path to the directory containing the images
dir_path = '/content/drive/MyDrive/Dataset/training'

# Set the desired size for the images
img_size = (224, 224)

# Loop through the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        img = cv2.imread(os.path.join(dir_path, filename))

        # Resize the image to the desired size
        resized_img = cv2.resize(img, img_size)

        # Save the resized image with the same filename and extension
        cv2.imwrite(os.path.join(dir_path, filename), resized_img)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Set the path to the directory containing the X-Ray images
dir_path = '/content/drive/MyDrive/Dataset/training'

# Set the desired size for the images
img_size = (224, 224)

# Initialize empty lists for the images and labels
images = []
labels = []

# Loop through the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        img = cv2.imread(os.path.join(dir_path, filename))

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Resize the image to the desired size
        resized_img = cv2.resize(gray, img_size)

        # Normalize the pixel values to be between 0 and 1
        normalized_img = resized_img / 255.0

        # Determine the label for the image based on the filename prefix
        if filename.startswith('Y_'):
            label = 1
        elif filename.startswith('N_'):
            label = 0
        else:
            label = -1

        # Add the image and label to the lists
        images.append(normalized_img)
        labels.append(label)

# Convert the lists of images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Add a channel dimension to the images array
images = np.expand_dims(images, axis=-1)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# Fit the ImageDataGenerator to the training data
datagen.fit(train_images)

# Shuffle the training data
idx = np.random.permutation(len(train_images))
train_images, train_labels = train_images[idx], train_labels[idx]

# One-hot encode the labels
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)

# Save the preprocessed data to disk
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)
np.save('val_images.npy', val_images)
np.save('val_labels.npy', val_labels)


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50

# Set the path to the directory containing the training images
train_dir = '/path/to/train'

# Set the desired size for the images
img_size = (224, 224)

# Set the batch size for training
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest', preprocessing_function=lambda x: x/255.0)

# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Add a global average pooling layer and a dense output layer for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

# Create the modified model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the weights of the base ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Create a generator for the training data
train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

# Train the model for a few epochs
model.fit(train_generator, epochs=1)

# Save the trained model
model.save('bone_fracture_detection.h5')


#Training


Import ResNet 50 model and set it up for Bone Fracture Detection

In [ ]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import pandas as pd

# Set the number of classes for the bone fracture detection task
num_classes = 2

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a dense output layer for the bone fracture detection task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create a new model with the ResNet50 base and the new output layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the ResNet50 base to prevent them from being trained
for layer in base_model.layers:
    layer.trainable = False

# Load the training data and labels
train_data = np.load('images_array.npy')
train_labels = pd.read_csv('image_labels.csv')

# Convert the labels to one-hot encoding
train_labels = keras.utils.to_categorical(train_labels['label'], num_classes)

# Compile the model with a categorical cross-entropy loss function and an Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data and labels
model.fit(train_data, train_labels, epochs=1, batch_size=32, validation_split=0.2)

#Testing

Input Image to check for Bone Fracture

#Push Changes to GitHub

Clone GitHub Repo to GDrive

In [ ]:
!git clone https://github.com/nandodeomkar/BFD-using-ResNet50 /content/drive/MyDrive/GitHub

Stage the Changes

In [ ]:
!git add .

Commit Changes

In [ ]:
!git config --global user.email "omkar.nandode1@gmail.com"
!git config --global user.name "NandodeOmkar"

In [ ]:
!git commit -m "Pseudo Code Arranges in the Project"

Push Changes